# Video "What is TensorFlow"

In [1]:
%config IPCompleter.greedy=True

In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.2.1


In [3]:
tf.reset_default_graph()
a = tf.placeholder(np.float32, (3, 4))
b = tf.Variable(tf.ones((4, 5)))
c = a @ b

In [4]:
a

<tf.Tensor 'Placeholder:0' shape=(3, 4) dtype=float32>

In [5]:
b

<tf.Variable 'Variable:0' shape=(4, 5) dtype=float32_ref>

In [6]:
c

<tf.Tensor 'matmul:0' shape=(3, 5) dtype=float32>

In [7]:
s = tf.InteractiveSession()

In [8]:
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((3, 4))}) #feed value of shape (3, 4) for Tensor 'Placeholder:0'

array([[ 4.,  4.,  4.,  4.,  4.],
       [ 4.,  4.,  4.,  4.,  4.],
       [ 4.,  4.,  4.,  4.,  4.]], dtype=float32)

In [9]:
s.close()

In [10]:
g = tf.Graph()
with g.as_default():
    pass

In [13]:
x = tf.placeholder(tf.float32, (None, 10)) #3 times

In [14]:
tf.get_default_graph().get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'ones' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'matmul' type=MatMul>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'Placeholder_2' type=Placeholder>,
 <tf.Operation 'Placeholder_3' type=Placeholder>]

In [15]:
tf.get_default_graph().get_operations()[0].outputs

[<tf.Tensor 'Placeholder:0' shape=(3, 4) dtype=float32>]

In [16]:
tf.reset_default_graph() #reset graph 

In [17]:
#initialization of variables
tf.get_variable("w", shape = (10, 20), dtype = tf.float32)

<tf.Variable 'w:0' shape=(10, 20) dtype=float32_ref>

# Video "Our first model in TensorFlow"

### Simple optimization (with simple prints)

In [18]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.1) #0.1 is learning rate
step = optimizer.minimize(f, var_list=[x])

In [20]:
tf.trainable_variables()

[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

In [21]:
with tf.Session() as s:  # in this way session will be closed automatically
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f])
        print(curr_x, curr_f)

-0.514561 0.413708
-0.411649 0.264773
-0.329319 0.169455
-0.263455 0.108451
-0.210764 0.0694088
-0.168611 0.0444216
-0.134889 0.0284298
-0.107911 0.0181951
-0.0863291 0.0116449
-0.0690633 0.00745271


### Simple optimization (with tf.Print)

In [22]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2
f = tf.Print(f, [x, f], "x, f:")

In [23]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [24]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(10):
        s.run([step, f])

In [25]:
# Prints to jupyter server stdout (not available in Coursera Notebooks):
# 2018-07-21 18:01:27.308270: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-1.0670249][1.1385423]
# 2018-07-21 18:01:27.308809: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.85361993][0.72866696]
# 2018-07-21 18:01:27.309116: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.68289596][0.46634689]
# 2018-07-21 18:01:27.309388: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.54631674][0.29846197]
# 2018-07-21 18:01:27.309678: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.43705338][0.19101566]
# 2018-07-21 18:01:27.309889: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.34964269][0.12225001]
# 2018-07-21 18:01:27.310213: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.27971417][0.078240015]
# 2018-07-21 18:01:27.310475: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.22377133][0.050073609]
# 2018-07-21 18:01:27.310751: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.17901707][0.032047112]
# 2018-07-21 18:01:27.310963: I tensorflow/core/kernels/logging_ops.cc:79] x, f:[-0.14321366][0.020510152]

### Simple optimization (with TensorBoard logging)

In [26]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [27]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [28]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all() #merge all these summaries in one single node

In [29]:
s = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter("logs/1", s.graph)
s.run(tf.global_variables_initializer())
for i in range(10):
    _, curr_summaries = s.run([step, summaries])
    summary_writer.add_summary(curr_summaries, i)
    summary_writer.flush()

Run  `tensorboard --logdir=./logs` in bash

This is what you can see in your browser **(not available in Coursera Notebooks)**
<img src="images/TensorBoard.jpg" style="width:70%">

**If you're running on Google Colab you can still run TensorBoard!**

In [ ]:
# !!! RUN THIS CELL ONLY ON GOOGLE COLAB !!!
! wget https://raw.githubusercontent.com/hse-aml/intro-to-dl/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab

# run tensorboard in background
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# expose port and show the link
setup_google_colab.expose_port_on_colab(6006)

In [30]:
s.close()

### Training a linear model

In [31]:
# generate model data
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[ 0.70478788  0.70610302  0.76007219]]


In [32]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D))
target = tf.placeholder(tf.float32, shape=(None, 1))

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights

loss = tf.reduce_mean((target - predictions) ** 2)

print(target.shape, predictions.shape, loss.shape)

(?, 1) (?, 1) ()


In [33]:
optimizer = tf.train.GradientDescentOptimizer(0.1) #run backpropagation and optimize all the variables
step = optimizer.minimize(loss)

In [34]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss) #print the current loss every 50 iterations

0.155084
0.0487086
0.0420832
0.0408654
0.0406411
0.0405996


In [35]:
# found weights
curr_weights.T

array([[ 0.71868843,  0.700041  ,  0.7711879 ]], dtype=float32)

In [36]:
# true weights
w.T

array([[ 0.70478788,  0.70610302,  0.76007219]])

In [37]:
s = tf.InteractiveSession()
saver = tf.train.Saver(tf.trainable_variables())
s.run(tf.global_variables_initializer())
for i in range(300):
    _, curr_loss, curr_weights = s.run([step, loss, weights], feed_dict={features: x, target: y})
    if i % 50 == 0:
        saver.save(s, "logs/2/model.ckpt", global_step=i) #save all work and continue if it's interrupted
print(curr_loss)

0.0405993


In [38]:
saver.last_checkpoints

['logs/2/model.ckpt-50',
 'logs/2/model.ckpt-100',
 'logs/2/model.ckpt-150',
 'logs/2/model.ckpt-200',
 'logs/2/model.ckpt-250']

In [39]:
saver.restore(s, "logs/2/model.ckpt-50")

INFO:tensorflow:Restoring parameters from logs/2/model.ckpt-50
